In [2]:
# Importando as bibliotecas
import re
import mysql.connector
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lower, to_timestamp, month, hour, datediff, dayofweek, count, when, isnan
from pyspark.sql.types import StringType
from pyspark.ml.feature import Tokenizer

## Carregando os dados
Nesta etapa, irei carregar os dados.

In [3]:
# Iniciando sessão no Spark
spark = SparkSession.builder.appName('Cluster').getOrCreate()


23/06/04 08:44:26 WARN Utils: Your hostname, daniel-VJFE43F11X-XXXXXX resolves to a loopback address: 127.0.1.1; using 192.168.0.157 instead (on interface wlo1)
23/06/04 08:44:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/04 08:44:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Lendos os dados csv
customers = spark.read.csv("../data/raw/olist_customers_dataset.csv", header=True)
geolocalization = spark.read.csv("../data/raw/olist_geolocation_dataset.csv", header=True)
order_items = spark.read.csv("../data/raw/olist_order_items_dataset.csv", header=True)
order_payments = spark.read.csv("../data/raw/olist_order_payments_dataset.csv", header=True)
order_reviews = spark.read.csv("../data/raw/olist_order_reviews_dataset.csv", header=True)
orders = spark.read.csv("../data/raw/olist_orders_dataset.csv", header=True)
products = spark.read.csv("../data/raw/olist_products_dataset.csv", header=True)
sellers = spark.read.csv("../data/raw/olist_sellers_dataset.csv", header=True)
product_category = spark.read.csv("../data/raw/product_category_name_translation.csv", header=True)

# Lendo os dados externos
ceps = spark.read.parquet("../data/external/cep_prefixes_processed.parquet")

## Transformando os dados
Nesta etapa, irei realizar uma série de transformações nas tabelas de forma individual, realizando tratamento específico para cada uma. Além disso, também irei gerar metadados, especificando o schema e descrição de cada coluna da tabela.

### Criando tabelas temporárias

In [5]:
# Criando tabelas temporárias
customers.createOrReplaceTempView("customers")
geolocalization.createOrReplaceTempView('geolocalization')
order_items.createOrReplaceTempView('order_items')
order_payments.createOrReplaceTempView('order_payments')
order_reviews.createOrReplaceTempView('order_reviews')
orders.createOrReplaceTempView('orders')
products.createOrReplaceTempView('products')
sellers.createOrReplaceTempView('sellers')
product_category.createOrReplaceTempView('product_category')
ceps.createOrReplaceTempView('ceps')

## customers

In [6]:
# Antes de decidir as mudanças, devemos checar as primeiras linhas
spark.sql("SELECT *  FROM customers LIMIT 5").show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                   09790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                   01151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                   08775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+



In [7]:
# Agora veremos o schema para verificar se o dado condiz com o seu tipo
customers.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [8]:
# Verificando se há inconsistencias nos valores unicos da coluna customer_state
customers.select('customer_state').distinct().collect()

[Row(customer_state='SC'),
 Row(customer_state='RO'),
 Row(customer_state='PI'),
 Row(customer_state='AM'),
 Row(customer_state='RR'),
 Row(customer_state='GO'),
 Row(customer_state='TO'),
 Row(customer_state='MT'),
 Row(customer_state='SP'),
 Row(customer_state='ES'),
 Row(customer_state='PB'),
 Row(customer_state='RS'),
 Row(customer_state='MS'),
 Row(customer_state='AL'),
 Row(customer_state='MG'),
 Row(customer_state='PA'),
 Row(customer_state='BA'),
 Row(customer_state='SE'),
 Row(customer_state='PE'),
 Row(customer_state='CE'),
 Row(customer_state='RN'),
 Row(customer_state='RJ'),
 Row(customer_state='MA'),
 Row(customer_state='AC'),
 Row(customer_state='DF'),
 Row(customer_state='PR'),
 Row(customer_state='AP')]

In [9]:
customers.select('customer_city').collect()

[Row(customer_city='franca'),
 Row(customer_city='sao bernardo do campo'),
 Row(customer_city='sao paulo'),
 Row(customer_city='mogi das cruzes'),
 Row(customer_city='campinas'),
 Row(customer_city='jaragua do sul'),
 Row(customer_city='sao paulo'),
 Row(customer_city='timoteo'),
 Row(customer_city='curitiba'),
 Row(customer_city='belo horizonte'),
 Row(customer_city='montes claros'),
 Row(customer_city='rio de janeiro'),
 Row(customer_city='lencois paulista'),
 Row(customer_city='sao paulo'),
 Row(customer_city='caxias do sul'),
 Row(customer_city='piracicaba'),
 Row(customer_city='rio de janeiro'),
 Row(customer_city='guarulhos'),
 Row(customer_city='sao paulo'),
 Row(customer_city='pacaja'),
 Row(customer_city='florianopolis'),
 Row(customer_city='aparecida de goiania'),
 Row(customer_city='sao paulo'),
 Row(customer_city='curitiba'),
 Row(customer_city='sao paulo'),
 Row(customer_city='santo andre'),
 Row(customer_city='goiania'),
 Row(customer_city='sao paulo'),
 Row(customer_city

Aparentemente, as colunas ``customer_state`` e ``customer_city`` necessitarão de
mudanças, onde a primeira receberá uma normalização de case e a segunda será 
substituída pela coluna de cidades da tabela externa, que não apresenta erros. 
Além disso, vale ressaltar que a coluna ``customer_zip_code_prefix``, mesmo 
apresentando valores numéricos, será mantida como **string**, pois ela perderia 
os zeros no início caso fosse alterada para formatos numéricos como **Int**. 


In [10]:
customers.columns

['customer_id',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state']

In [11]:
# Tratando a tabela customers
customers = spark.sql("SELECT cs.customer_id,  \
                        cs.customer_unique_id, \
                        cs.customer_zip_code_prefix, \
                        LOWER(cp.locality) AS customer_city,  \
                        LOWER(cs.customer_state) AS customer_state \
                       FROM customers cs  \
                        LEFT JOIN ceps cp  \
                        ON CAST(cs.customer_zip_code_prefix AS INT) = cp.cep_prefix")

In [12]:
# Verificando o resultado
customers.show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            sp|
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            sp|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                   09790|são bernardo do c...|            sp|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                   09790|são bernardo do c...|            sp|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                   01151|           são paulo|            sp|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                   01151|           são paulo|            sp|
|b2b6027bc5c5109e5...|259dac757896d24d7...|            

## geolocalization

In [13]:
# Irei verificar as primeiras linhas e o schema novamente
geolocalization.show()

+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|                      01037| -23.54562128115268| -46.63929204800168|       sao paulo|               SP|
|                      01046|-23.546081127035535| -46.64482029837157|       sao paulo|               SP|
|                      01046| -23.54612896641469| -46.64295148361138|       sao paulo|               SP|
|                      01041|  -23.5443921648681| -46.63949930627844|       sao paulo|               SP|
|                      01035|-23.541577961711493| -46.64160722329613|       sao paulo|               SP|
|                      01012|-23.547762303364266| -46.63536053788448|       são paulo|               SP|
|                      01047|-23.546273112412678| -46.6

In [14]:
geolocalization.printSchema()

root
 |-- geolocation_zip_code_prefix: string (nullable = true)
 |-- geolocation_lat: string (nullable = true)
 |-- geolocation_lng: string (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)



In [15]:
# Verificando se há inconsistencias nos valores unicos da coluna geolocation_state
geolocalization.select('geolocation_state').distinct().collect()

[Row(geolocation_state='SP'),
 Row(geolocation_state='RN'),
 Row(geolocation_state='AC'),
 Row(geolocation_state='ES'),
 Row(geolocation_state='RJ'),
 Row(geolocation_state='MG'),
 Row(geolocation_state='PI'),
 Row(geolocation_state='AM'),
 Row(geolocation_state='RR'),
 Row(geolocation_state='GO'),
 Row(geolocation_state='PB'),
 Row(geolocation_state='AL'),
 Row(geolocation_state='PA'),
 Row(geolocation_state='BA'),
 Row(geolocation_state='SE'),
 Row(geolocation_state='PE'),
 Row(geolocation_state='CE'),
 Row(geolocation_state='MA'),
 Row(geolocation_state='DF'),
 Row(geolocation_state='AP'),
 Row(geolocation_state='SC'),
 Row(geolocation_state='RO'),
 Row(geolocation_state='TO'),
 Row(geolocation_state='MT'),
 Row(geolocation_state='RS'),
 Row(geolocation_state='MS'),
 Row(geolocation_state='PR')]

In [16]:
# Verificando se há inconsistência nos valores únicos da coluna geolocation_city
geolocalization.select('geolocation_city').distinct().collect()

[Row(geolocation_city='sao bernardo do campo'),
 Row(geolocation_city='são paulo'),
 Row(geolocation_city='sãopaulo'),
 Row(geolocation_city='sao jose dos campos'),
 Row(geolocation_city='jundiaí'),
 Row(geolocation_city='sp'),
 Row(geolocation_city='sao paulo'),
 Row(geolocation_city='taboão da serra'),
 Row(geolocation_city='sa£o paulo'),
 Row(geolocation_city='redencao da serra'),
 Row(geolocation_city='itanhaem'),
 Row(geolocation_city='carapicuiba'),
 Row(geolocation_city='sao francisco da praia'),
 Row(geolocation_city='taubaté'),
 Row(geolocation_city='iguape'),
 Row(geolocation_city='itapecerica da serra'),
 Row(geolocation_city='santos'),
 Row(geolocation_city='franco da rocha'),
 Row(geolocation_city='cotia'),
 Row(geolocation_city='suzano'),
 Row(geolocation_city='embu guaçu'),
 Row(geolocation_city='caruara'),
 Row(geolocation_city='mogi das cruzes'),
 Row(geolocation_city='cananeia'),
 Row(geolocation_city='salesopolis'),
 Row(geolocation_city='jordanesia'),
 Row(geolocati

Aqui temos uma situação semelhante a da tabela anterior, sendo necessário 
normalizar a coluna ``geolocation_state``, mas com adições, onde as colunas que
representam dados geográficos também terão seu tipo alterado. Além disso, a 
coluna ``geolocation_city``apresentou alguns valores duplicados, mas que se 
referem a mesma cidade, então será necessário realizar uma limpeza nela.

In [17]:
geolocalization = spark.sql("SELECT g.geolocation_zip_code_prefix,  \
                          CAST(g.geolocation_lat AS DOUBLE),  \
                          CAST(g.geolocation_lng AS DOUBLE),  \
                          LOWER(cp.locality) AS geolocation_city, \
                          g.geolocation_city AS geolocation_city_original, \
                          LOWER(geolocation_state) AS geolocation_state \
                         FROM geolocalization AS g  \
                         LEFT JOIN ceps cp  \
                         ON CAST(g.geolocation_zip_code_prefix AS INT) = cp.cep_prefix")

In [18]:
# Verificando os valores missing
geolocalization.select([count(when(isnan(coluna) | col(coluna).isNull(), coluna)).alias(coluna) for coluna in geolocalization.columns]
   ).show()

+---------------------------+---------------+---------------+----------------+-------------------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat|geolocation_lng|geolocation_city|geolocation_city_original|geolocation_state|
+---------------------------+---------------+---------------+----------------+-------------------------+-----------------+
|                          0|              0|              0|            3676|                        0|                0|
+---------------------------+---------------+---------------+----------------+-------------------------+-----------------+



In [19]:
# Como a coluna geolocation_city_CEP possui valores missing, terei que inputar 
# os valores originais da coluna geolocation_city. Para isso, irei dividir a
# tabela em duas, uma com os valores missing e outra com os valores não missing
# e depois concatená-las

geolocalization_com_missing = geolocalization.filter("geolocation_city IS NULL")
geolocalization = geolocalization.filter("geolocation_city IS NOT NULL")
geolocalization_com_missing = geolocalization_com_missing.withColumn('geolocation_city', col("geolocation_city_original"))
geolocalization = geolocalization.union(geolocalization_com_missing)

# Também será necessário dropar a coluna original, que contém inconsistencias 
geolocalization = geolocalization.drop(col('geolocation_city_original'))

## order_items

In [20]:
# Antes de decidi as mudanças, devemos checar as primeiras linhas e o schema
order_items.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.90|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.90|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.00|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.90|        18.14|
+--------------------+-------------+----

In [21]:
order_items.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- freight_value: string (nullable = true)



In [22]:
order_items.select('shipping_limit_date').distinct().collect()

[Row(shipping_limit_date='2018-05-18 04:52:44'),
 Row(shipping_limit_date='2017-08-04 13:55:22'),
 Row(shipping_limit_date='2017-04-27 05:10:46'),
 Row(shipping_limit_date='2018-06-22 12:31:17'),
 Row(shipping_limit_date='2017-11-03 17:06:08'),
 Row(shipping_limit_date='2017-10-05 02:28:20'),
 Row(shipping_limit_date='2017-09-11 14:43:50'),
 Row(shipping_limit_date='2018-03-09 20:30:52'),
 Row(shipping_limit_date='2018-08-13 17:35:18'),
 Row(shipping_limit_date='2017-09-08 18:30:21'),
 Row(shipping_limit_date='2018-01-18 20:09:35'),
 Row(shipping_limit_date='2017-08-31 21:55:06'),
 Row(shipping_limit_date='2017-10-30 03:49:38'),
 Row(shipping_limit_date='2017-04-02 22:45:16'),
 Row(shipping_limit_date='2018-03-05 16:36:05'),
 Row(shipping_limit_date='2017-11-03 11:48:19'),
 Row(shipping_limit_date='2017-01-21 14:50:19'),
 Row(shipping_limit_date='2018-03-01 20:08:16'),
 Row(shipping_limit_date='2018-07-13 13:50:17'),
 Row(shipping_limit_date='2017-09-15 01:44:00'),
 Row(shipping_limit_

Para essa tabela, irei alterar o tipo de dado das colunas ``price`` e 
``freight_value`` para **Float** e ``shipping_limit_date``para **timestamp**.

In [23]:
order_items = spark.sql("SELECT order_id, \
                          order_item_id, \
                          product_id, \
                          seller_id, \
                          CAST(shipping_limit_date AS TIMESTAMP), \
                          CAST(price AS FLOAT), \
                          CAST(freight_value AS FLOAT)  \
                         FROM order_items")

## order_payments

In [24]:
# Vamos checar as primeiras linhas e o schema
order_payments.show(5)

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
+--------------------+------------------+------------+--------------------+-------------+
only showing top 5 rows



In [25]:
order_payments.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: string (nullable = true)
 |-- payment_value: string (nullable = true)



In [26]:
# Agora irei verificar os valores distintos da coluna payment_type, 
# payment_sequential e payment_installments
order_payments.select('payment_type').distinct().collect()

[Row(payment_type='boleto'),
 Row(payment_type='not_defined'),
 Row(payment_type='credit_card'),
 Row(payment_type='voucher'),
 Row(payment_type='debit_card')]

In [27]:
order_payments.select('payment_sequential').distinct().collect()

[Row(payment_sequential='7'),
 Row(payment_sequential='15'),
 Row(payment_sequential='11'),
 Row(payment_sequential='29'),
 Row(payment_sequential='3'),
 Row(payment_sequential='8'),
 Row(payment_sequential='22'),
 Row(payment_sequential='28'),
 Row(payment_sequential='16'),
 Row(payment_sequential='5'),
 Row(payment_sequential='18'),
 Row(payment_sequential='27'),
 Row(payment_sequential='17'),
 Row(payment_sequential='26'),
 Row(payment_sequential='6'),
 Row(payment_sequential='19'),
 Row(payment_sequential='23'),
 Row(payment_sequential='25'),
 Row(payment_sequential='9'),
 Row(payment_sequential='24'),
 Row(payment_sequential='1'),
 Row(payment_sequential='10'),
 Row(payment_sequential='4'),
 Row(payment_sequential='12'),
 Row(payment_sequential='13'),
 Row(payment_sequential='14'),
 Row(payment_sequential='21'),
 Row(payment_sequential='2'),
 Row(payment_sequential='20')]

In [28]:
order_payments.select('payment_installments').distinct().collect()

[Row(payment_installments='7'),
 Row(payment_installments='15'),
 Row(payment_installments='11'),
 Row(payment_installments='3'),
 Row(payment_installments='8'),
 Row(payment_installments='22'),
 Row(payment_installments='16'),
 Row(payment_installments='0'),
 Row(payment_installments='5'),
 Row(payment_installments='18'),
 Row(payment_installments='17'),
 Row(payment_installments='6'),
 Row(payment_installments='9'),
 Row(payment_installments='24'),
 Row(payment_installments='1'),
 Row(payment_installments='20'),
 Row(payment_installments='10'),
 Row(payment_installments='4'),
 Row(payment_installments='12'),
 Row(payment_installments='13'),
 Row(payment_installments='14'),
 Row(payment_installments='21'),
 Row(payment_installments='2'),
 Row(payment_installments='23')]

Para essa tabela, irei alterar o tipo de dado das colunas ``payment_sequential``
e ``payment_installments`` para **Int** e ``payment_value`` para **Float**.

In [29]:
order_payments = spark.sql("SELECT order_id, \
                             CAST(payment_sequential AS INT), \
                             payment_type, \
                             CAST(payment_installments AS INT), \
                             CAST(payment_value AS FLOAT)  \
                            FROM order_payments")

## order_reviews

In [30]:
# Vamos checar as primeiras linhas e o schema
order_reviews.show(5)

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                null|                  null| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                null|                  null| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                null|                  null| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                null|  Recebi bem antes ...| 2017-04-21 00:00:00|   

In [31]:
order_reviews.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)



In [32]:
# Agora irei checar os valores distintos da coluna review_score, review_create_date e review_answer_timestamp
order_reviews.select('review_score').distinct().collect()

[Row(review_score='2018-01-27 20:03:09'),
 Row(review_score='2017-07-09 00:33:40'),
 Row(review_score='2017-10-21 01:12:49'),
 Row(review_score='2018-04-16 19:34:18'),
 Row(review_score='2017-03-28 22:38:46'),
 Row(review_score='2018-05-04 02:18:31'),
 Row(review_score='2017-12-14 20:09:47'),
 Row(review_score='2018-01-30 16:02:47'),
 Row(review_score='2018-01-04 17:32:51'),
 Row(review_score=' alias comprei justamente pelo prazo de entrega."'),
 Row(review_score='2018-06-17 20:29:17'),
 Row(review_score='2017-02-10 03:19:21'),
 Row(review_score='2018-05-04 22:37:24'),
 Row(review_score='2017-04-21 00:00:00'),
 Row(review_score='2017-03-19 22:44:11'),
 Row(review_score='2018-08-30 23:20:15'),
 Row(review_score='2018-09-03 11:26:24'),
 Row(review_score=' fica tudo engovinhado gostaria de trocar produto.'),
 Row(review_score='2017-10-08 20:42:50'),
 Row(review_score='2016-10-28 12:21:53'),
 Row(review_score='2018-04-01 00:27:51'),
 Row(review_score='2018-03-14 11:39:53'),
 Row(review_sco

In [33]:
order_reviews.select('review_comment_message', 'review_creation_date').where("review_creation_date LIKE 'veio%'").show()

+----------------------+--------------------+
|review_comment_message|review_creation_date|
+----------------------+--------------------+
|  "a capinha a qual...|veio bem embalada...|
+----------------------+--------------------+



In [34]:
order_reviews.select('review_creation_date').distinct().collect()

[Row(review_creation_date='2017-05-08 00:00:00'),
 Row(review_creation_date='2017-09-03 00:00:00'),
 Row(review_creation_date='2017-04-21 00:00:00'),
 Row(review_creation_date='2017-10-27 00:00:00'),
 Row(review_creation_date='2018-05-16 00:00:00'),
 Row(review_creation_date='2017-05-18 00:00:00'),
 Row(review_creation_date='2017-04-18 00:00:00'),
 Row(review_creation_date='2017-02-22 00:00:00'),
 Row(review_creation_date='2018-03-28 00:00:00'),
 Row(review_creation_date='2017-08-26 00:00:00'),
 Row(review_creation_date='2018-01-03 00:00:00'),
 Row(review_creation_date='2017-05-12 00:00:00'),
 Row(review_creation_date='veio bem embalada oque indica que foi embalada ja quebrada.triste"'),
 Row(review_creation_date='2016-12-04 00:00:00'),
 Row(review_creation_date=' com isso tive que cancelar o cartão'),
 Row(review_creation_date='2017-12-19 00:00:00'),
 Row(review_creation_date='2017-05-13 00:00:00'),
 Row(review_creation_date='2018-06-30 00:00:00'),
 Row(review_creation_date='2017-03-0

In [35]:
order_reviews.select('review_answer_timestamp').distinct().collect()

[Row(review_answer_timestamp='2018-05-25 03:45:28'),
 Row(review_answer_timestamp='2018-05-10 17:19:17'),
 Row(review_answer_timestamp='2018-02-06 11:51:18'),
 Row(review_answer_timestamp='2018-08-22 18:33:02'),
 Row(review_answer_timestamp='2017-11-06 22:29:34'),
 Row(review_answer_timestamp='2018-02-26 13:59:17'),
 Row(review_answer_timestamp='2017-11-05 10:43:54'),
 Row(review_answer_timestamp='2018-04-21 03:31:59'),
 Row(review_answer_timestamp='2018-01-26 22:29:43'),
 Row(review_answer_timestamp='2017-05-22 12:05:04'),
 Row(review_answer_timestamp='2018-08-11 21:20:30'),
 Row(review_answer_timestamp='2018-08-02 12:20:27'),
 Row(review_answer_timestamp='2018-05-21 02:29:27'),
 Row(review_answer_timestamp='2017-12-24 20:07:54'),
 Row(review_answer_timestamp='2017-05-01 12:44:11'),
 Row(review_answer_timestamp='2017-12-07 09:43:36'),
 Row(review_answer_timestamp='2017-07-23 21:36:45'),
 Row(review_answer_timestamp='2018-02-09 10:19:32'),
 Row(review_answer_timestamp='2017-11-03 07:01

A tabela apresenta um erro que comprometeu todas as colunas. A escolha do 
delimitador para as colunas foi a vírgula, mas alguns valores da coluna 
``review_comment_message`` apresentam vírgulas, o que fez com que  os valores 
fossem deslocados para a direita. Para resolver esse problema, o ponto e vírgula
deveria ter sido usado ao invés da vírgula, pois no estado atual, o arquivo não
possui muita utilidade. A Tabela será ignorada.

## orders

In [42]:
# Irei verificar os valores e o schema
orders.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [43]:
orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_carrier_date: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)
 |-- order_estimated_delivery_date: string (nullable = true)



In [46]:
# Verificando os valores distintos da coluna order_status
orders.select('order_status').distinct().collect()

[Row(order_status='shipped'),
 Row(order_status='canceled'),
 Row(order_status='invoiced'),
 Row(order_status='created'),
 Row(order_status='delivered'),
 Row(order_status='unavailable'),
 Row(order_status='processing'),
 Row(order_status='approved')]

In [47]:
orders.columns

['order_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date']

A coluna em questão necessita apenas de alteração de tipo de variável, onde
``order_purchase_timestamp``, ``order_approved_at``, 
``order_delivered_carrier_date`` e ``order_delivered_customer_date`` serão 
alteradas para **timestamp** e ``order_estimated_delivery_date`` para **date**.

In [49]:
orders = spark.sql("SELECT order_id,  \
                     customer_id,  \
                     order_status,  \
                     CAST(order_purchase_timestamp AS TIMESTAMP),  \
                     CAST(order_approved_at AS TIMESTAMP),  \
                     CAST(order_delivered_carrier_date AS TIMESTAMP),  \
                     CAST(order_delivered_customer_date AS TIMESTAMP),  \
                     CAST(order_estimated_delivery_date AS DATE)  \
                    FROM orders")

## products

In [51]:
# Checando os valores e o schema
products.show(5)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [52]:
products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: string (nullable = true)
 |-- product_description_lenght: string (nullable = true)
 |-- product_photos_qty: string (nullable = true)
 |-- product_weight_g: string (nullable = true)
 |-- product_length_cm: string (nullable = true)
 |-- product_height_cm: string (nullable = true)
 |-- product_width_cm: string (nullable = true)



In [53]:
# Checando os valores distintos da coluna product_category_name
products.select('product_category_name').distinct().collect()

[Row(product_category_name='pcs'),
 Row(product_category_name='bebes'),
 Row(product_category_name='artes'),
 Row(product_category_name='cine_foto'),
 Row(product_category_name='moveis_decoracao'),
 Row(product_category_name='pc_gamer'),
 Row(product_category_name='construcao_ferramentas_construcao'),
 Row(product_category_name='tablets_impressao_imagem'),
 Row(product_category_name='fashion_roupa_masculina'),
 Row(product_category_name='artigos_de_festas'),
 Row(product_category_name='artigos_de_natal'),
 Row(product_category_name='la_cuisine'),
 Row(product_category_name='flores'),
 Row(product_category_name='livros_tecnicos'),
 Row(product_category_name=None),
 Row(product_category_name='telefonia_fixa'),
 Row(product_category_name='construcao_ferramentas_seguranca'),
 Row(product_category_name='cool_stuff'),
 Row(product_category_name='eletrodomesticos'),
 Row(product_category_name='livros_importados'),
 Row(product_category_name='pet_shop'),
 Row(product_category_name='casa_constr

In [55]:
products.columns

['product_id',
 'product_category_name',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm']

A tabela irá precisar de uma mudança no tipo de algumas colunas, onde ``product_name_lenght``, ``product_description_lenght``, ``product_photos_qty``, ``product_weight_g``, ``product_length_cm``, ``product_height_cm`` e ``product_width_cm`` serão alteradas para **Int**. 

In [56]:
products = spark.sql("SELECT product_id,  \
                        LOWER(product_category_name) AS product_category_name,  \
                        CAST(product_name_lenght AS INT),  \
                        CAST(product_description_lenght AS INT),  \
                        CAST(product_photos_qty AS INT),  \
                        CAST(product_weight_g AS INT),  \
                        CAST(product_length_cm AS INT),  \
                        CAST(product_height_cm AS INT),  \
                        CAST(product_width_cm AS INT)  \
                      FROM products")

## sellers

In [74]:
# Verificando as primeiras linhas e o schema
sellers.show(5)


+--------------------+----------------------+-----------+------------+
|           seller_id|seller_zip_code_prefix|seller_city|seller_state|
+--------------------+----------------------+-----------+------------+
|4be2e7f96b4fd749d...|                 69900| rio branco|          ac|
|4be2e7f96b4fd749d...|                 69900| rio branco|          ac|
|327b89b872c14d1c0...|                 69005|     manaus|          am|
|327b89b872c14d1c0...|                 69005|     manaus|          am|
|43753b27d77860f16...|                 44895| barro alto|          ba|
+--------------------+----------------------+-----------+------------+
only showing top 5 rows



In [62]:
sellers.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: string (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)



In [63]:
# Verificando os valores distintos da coluna seller_city
sellers.select('seller_city').distinct().collect()

[Row(seller_city='igrejinha'),
 Row(seller_city='brusque'),
 Row(seller_city='buritama'),
 Row(seller_city='carapicuiba'),
 Row(seller_city='garca'),
 Row(seller_city='sao joao de meriti'),
 Row(seller_city='fernando prestes'),
 Row(seller_city='ipaussu'),
 Row(seller_city='jacutinga'),
 Row(seller_city='nova friburgo'),
 Row(seller_city='araras'),
 Row(seller_city='sao pedro da aldeia'),
 Row(seller_city='santos'),
 Row(seller_city='itapecerica da serra'),
 Row(seller_city='ibitinga'),
 Row(seller_city='muqui'),
 Row(seller_city='cuiaba'),
 Row(seller_city='franco da rocha'),
 Row(seller_city='cotia'),
 Row(seller_city='marilia'),
 Row(seller_city='votorantim'),
 Row(seller_city='centro'),
 Row(seller_city='rio grande'),
 Row(seller_city='suzano'),
 Row(seller_city='mogi das cruses'),
 Row(seller_city='conselheiro lafaiete'),
 Row(seller_city='itabira'),
 Row(seller_city='osvaldo cruz'),
 Row(seller_city='mucambo'),
 Row(seller_city='venancio aires'),
 Row(seller_city='rio de janeiro,

A tabela necessita de normnalização na coluna ``seller_state`` e limpeza na 
coluna ``seller_city``, que contém inconsistências.

In [66]:
# Normalizando os valores da coluna seller_state
sellers = spark.sql("SELECT s.seller_id,  \
                        s.seller_zip_code_prefix,  \
                        LOWER(cp.locality) AS seller_city,  \
                        LOWER(s.seller_city) AS seller_city_original,  \
                        LOWER(seller_state) AS seller_state  \
                     FROM sellers  s\
                     LEFT JOIN ceps cp  \
                     ON CAST(s.seller_zip_code_prefix AS INT) = cp.cep_prefix")

In [73]:
# Agora irei ver se o JOIN gerou algum valor missing
sellers.select([count(when(isnan(coluna) | col(coluna).isNull(), coluna)).alias(coluna) for coluna in sellers.columns]).show()

+---------+----------------------+-----------+------------+
|seller_id|seller_zip_code_prefix|seller_city|seller_state|
+---------+----------------------+-----------+------------+
|        0|                     0|          0|           0|
+---------+----------------------+-----------+------------+



In [72]:
# Como a nova coluna sellers_city possui valores missing, terei que inputar 
# os valores originais da coluna sellers_city onde há falta. Para isso, irei 
# dividir a tabela em duas, uma com os valores missing e outra com os valores 
# não missing e depois concatená-las

sellers_com_missing = sellers.filter("seller_city IS NULL")
sellers = sellers.filter("seller_city IS NOT NULL")
sellers_com_missing = sellers_com_missing.withColumn('seller_city', col("seller_city_original"))
sellers = sellers.union(sellers_com_missing)

# Também será necessário dropar a coluna original, que contém inconsistencias 
sellers = sellers.drop(col('seller_city_original'))

## product_category

In [75]:
# Verificando as primeiras linhas e o schema
product_category.show()

+---------------------+-----------------------------+
|product_category_name|product_category_name_english|
+---------------------+-----------------------------+
|         beleza_saude|                health_beauty|
| informatica_acess...|         computers_accesso...|
|           automotivo|                         auto|
|      cama_mesa_banho|               bed_bath_table|
|     moveis_decoracao|              furniture_decor|
|        esporte_lazer|               sports_leisure|
|           perfumaria|                    perfumery|
| utilidades_domest...|                   housewares|
|            telefonia|                    telephony|
|   relogios_presentes|                watches_gifts|
|    alimentos_bebidas|                   food_drink|
|                bebes|                         baby|
|            papelaria|                   stationery|
| tablets_impressao...|         tablets_printing_...|
|           brinquedos|                         toys|
|       telefonia_fixa|     

In [76]:
product_category.printSchema()

root
 |-- product_category_name: string (nullable = true)
 |-- product_category_name_english: string (nullable = true)



In [77]:
# verificando valores unicos
product_category.select('product_category_name').distinct().collect()

[Row(product_category_name='pcs'),
 Row(product_category_name='bebes'),
 Row(product_category_name='artes'),
 Row(product_category_name='cine_foto'),
 Row(product_category_name='moveis_decoracao'),
 Row(product_category_name='tablets_impressao_imagem'),
 Row(product_category_name='construcao_ferramentas_construcao'),
 Row(product_category_name='fashion_roupa_masculina'),
 Row(product_category_name='artigos_de_festas'),
 Row(product_category_name='artigos_de_natal'),
 Row(product_category_name='la_cuisine'),
 Row(product_category_name='flores'),
 Row(product_category_name='livros_tecnicos'),
 Row(product_category_name='telefonia_fixa'),
 Row(product_category_name='construcao_ferramentas_seguranca'),
 Row(product_category_name='cool_stuff'),
 Row(product_category_name='eletrodomesticos'),
 Row(product_category_name='livros_importados'),
 Row(product_category_name='pet_shop'),
 Row(product_category_name='casa_construcao'),
 Row(product_category_name='livros_interesse_geral'),
 Row(product

A tabela em questão não apresenta problemas.

## Criando novas features

Agora que a tabela já parece estar boa, irei criar uma novas features. A primeira 
será criada a partir das colunas ``customer_state`` e ``seller_state``, criando categorias para os 
estados, onde cada estado vai receber a região em que se localiza no Brasil.

In [ ]:
# Criando lista de regiões  
estados_norte = ['AC', 'AM', 'RO', 'RR', 'AP', 'TO', 'PA']
estados_nordeste = ['MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'BA', 'SE']
estados_centro = ['MT', 'MS', 'DF', 'GO']
estados_sudeste = ['MG', 'SP', 'ES', 'RJ']
estados_sul = ['PR', 'SC', 'RS']

# Criando função 
func = lambda estado: 'NORTE' if estado in estados_norte else \
        ('NORDESTE' if estado in estados_nordeste else \
        ('CENTRO-OESTE' if estado in estados_centro else \
        ('SUDESTE' if estado in estados_sudeste else 'SUL')))

func_udf = udf(func, StringType())

In [ ]:
# Aplicando a função
orders_full = orders_full.withColumn("customer_region", func_udf("customer_state"))
orders_full = orders_full.withColumn("seller_region", func_udf("seller_state"))

Agora, irei criar uma nova coluna para resumir as categorias da coluna
``product_category_name``.

In [ ]:
# Criando novas categorias
eletronicos_e_tecnologia = ['pcs', 'pc_gamer', 'tablets_impressao_imagem', 
                            'telefonia_fixa', 'telefonia', 
                            'informatica_acessorios', 'eletronicos', 'audio', 
                            'consoles_games', 'dvds_blu_ray']

moda_e_acessorios = ['fashion_roupa_masculina', 'fashion_roupa_feminina',
                     'fashion_roupa_infanto_juvenil',
                     'fashion_underwear_e_moda_praia',
                     'fashion_bolsas_e_acessorios', 'fashion_calcados',
                     'fashion_esporte']

casa_e_decoracao = ['moveis_decoracao', 'moveis_colchao_e_estofado', 
                    'moveis_cozinha_area_de_servico_jantar_e_jardim', 
                    'moveis_quarto', 'moveis_sala', 'utilidades_domesticas', 
                    'cama_mesa_banho', 'casa_construcao', 'casa_conforto', 
                    'casa_conforto_2']

livros_e_educacao = ['livros_tecnicos', 'livros_importados', 
                     'livros_interesse_geral']

beleza_e_saude = ['beleza_saude', 'fraldas_higiene', 'perfumaria']

brinquedos_e_jogos = ['brinquedos', 'jogos', 'instrumentos_musicais']

alimentos_e_bebidas = ['alimentos_bebidas', 'bebidas', 'alimentos']

artigos_para_festas = ['artigos_de_festas', 'artigos_de_natal']

arte_e_artesanato = ['artes', 'cine_foto', 'artes_e_artesanato']

ferramentas_e_construcao = ['construcao_ferramentas_construcao', 
                            'construcao_ferramentas_seguranca', 
                            'construcao_ferramentas_jardim', 
                            'construcao_ferramentas_iluminacao', 
                            'construcao_ferramentas_ferramentas']

esporte_e_lazer = ['esporte_lazer']

outros = ['cool_stuff', 'flores', 'industria_comercio_e_negocios', 
          'malas_acessorios', 'seguros_e_servicos', 'market_place', 
          'relogios_presentes', 'papelaria', 'climatizacao', 
          'sinalizacao_e_seguranca', 'agro_industria_e_comercio', 
          'cds_dvds_musicais', 'musica', 'eletroportateis']

In [ ]:
# Criando a função
func = lambda categoria: 'eletronicos e tecnologia' if categoria in eletronicos_e_tecnologia else \
       ('moda e acessorios' if categoria in moda_e_acessorios else \
       ('casa e decoracao' if categoria in casa_e_decoracao else \
       ('livros e educacao' if categoria in livros_e_educacao else \
       ('beleza e saude' if categoria in beleza_e_saude else \
       ('brinquedos e jogos' if categoria in brinquedos_e_jogos else \
       ('alimentos_e_bebidas' if categoria in alimentos_e_bebidas else \
       ('artigos para festas' if categoria in artigos_para_festas else \
       ('arte e artesanato' if categoria in arte_e_artesanato else \
       ('ferramentas e construcao' if categoria in ferramentas_e_construcao else \
       ('esporte e lazer' if categoria in esporte_e_lazer else 'outros'))))))))))
       
func_udf = udf(func, StringType())


In [ ]:
# Criando nova coluna
orders_full = orders_full.withColumn('sub_category_product_name', func_udf('product_category_name'))

Como novas colunas, irei criar novas unidades de medida a partir  das 
unidades das colunas ``product_weight_g``, ``product_length_cm``, 
``product_height_cm`` e ``product_width_cm``.

In [ ]:
# Alterando a unidade de medida das colunas
orders_full = orders_full.withColumn('product_weight_kg', col('product_weight_g')/1000)
orders_full = orders_full.withColumn('product_length_m', col('product_length_cm')/100)
orders_full = orders_full.withColumn('product_height_m', col('product_height_cm')/100)
orders_full = orders_full.withColumn('product_width_m', col('product_width_cm')/100)


E para finalizar, irei extrair novas colunas a partir dos dados que representam datas.

In [ ]:
# Obtendo o dia da semana da compra
orders_full = orders_full.withColumn('order_day_of_week', dayofweek('order_purchase_timestamp'))

# Criando uma função para obter o nome do dia da semana
func = lambda dia: 'sunday' if dia == 1 else  \
        ('monday' if dia == 2 else  \
        ('tuesday' if dia == 3 else  \
        ('wednesday' if dia == 4 else  \
        ('thursday' if dia == 5 else  \
        ('friday' if dia == 6 else 'saturday')))))

func_udf = udf(func, StringType())

# Aplicando a função
orders_full = orders_full.withColumn('order_name_day_of_week', func_udf('order_day_of_week'))

In [ ]:
# Obtendo o mês da compŕa
orders_full = orders_full.withColumn('order_month', month('order_purchase_timestamp'))

# Criando uma função para nomear os meses
func = lambda mes: 'january' if mes == 1 else  \
        ('february' if mes == 2 else  \
        ('march' if mes == 3 else  \
        ('april' if mes == 4 else  \
        ('may' if mes == 5 else  \
        ('june' if mes == 6 else  \
        ('july' if mes == 7 else  \
        ('august' if mes == 8 else  \
        ('september' if mes == 9 else  \
        ('october' if mes == 10 else  \
        ('november' if mes == 11 else 'december'))))))))))
        
func_udf = udf(func, StringType())

# Aplicando a função
orders_full = orders_full.withColumn('order_name_month', func_udf('order_month'))

In [ ]:
# Obtendo a hora da compra
orders_full = orders_full.withColumn('order_hour', hour('order_purchase_timestamp'))

In [ ]:
# Obtendo o tempo de processamento do pedido
orders_full = orders_full.withColumn('order_processing_days', datediff('order_approved_at', 'order_purchase_timestamp'))

In [ ]:
# Calculando a diferença de dias entre a data do pedido e a data de entrega
orders_full = orders_full.withColumn('order_delivery_days', datediff('order_delivered_customer_date', 'order_purchase_timestamp'))

## Salvando os dados